## 専門知識労働者
- 保険テクノロジー企業である Insurellm の従業員が使用する、専門知識を持つナレッジ ワーカーである質問応答エージェント。エージェントは正確である必要があり、ソリューションは低コストである必要があります。
- このプロジェクトでは、RAG（検索拡張生成）を使用して、質問/回答アシスタントが高い精度を確保します。ココ、4つめの実装では、RAGをChatインターフェイスから試行します。

In [1]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr

In [2]:
# langchain、Chroma のインポート

# langchain
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

# VDBサポート (langchain_chroma)
from langchain_chroma import Chroma

In [3]:
# 価格は重要な要素（と言う建付け）なので、低コストモデルを採用
MODEL = "gpt-4o-mini"

# コレはVDBのChromaのDB名
db_name = "vector_db"

In [4]:
# .envファイルから環境変数をロード
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [5]:
# langChain のローダーを使用してKBのすべてのサブフォルダ内のすべてのドキュメントを読み取りリスト化
# 余談：メタデータは、DirectoryLoader および TextLoader によって生成され、そこにカスタムの属性、doc_typeを追加している。

folders = glob.glob("knowledge-base/*")

# 一部のユーザーに必要な修正を提供してくれた、コース受講生のCGとJon Rに感謝します。
text_loader_kwargs = {'encoding': 'utf-8'}
# それでもうまくいかない場合は、Windowsユーザーの中には次の行のコメントを解除する必要があるかもしれません。
# text_loader_kwargs={'autodetect_encoding': True}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [6]:
# テキストを200文字の重複部分を持たせた1000文字ごとのチャンク（かたまり）に分割
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# document も chunk も LangChain の Document クラス
chunks = text_splitter.split_documents(documents)

Created a chunk of size 1088, which is longer than the specified 1000


In [7]:
# 各チャンクに埋込ベクトルを関連付けるVDBに格納
# Chroma は SQLite ベースの人気のOSS VDB

embeddings = OpenAIEmbeddings()

# Huggingfaceのフリーのベクトル埋め込みを使用したい場合
# 次と、embeddings = OpenAIEmbeddings() を交換します：

# from langchain.embeddings import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Chroma DataStoreがすでに存在するかどうかを確認してください - もしそうなら、コレクションを削除してゼロから始める

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# Chroma VectorStoreを作成してください！

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 123 documents


## ベクトルストアの可視化
コードや表示される内容に変更はないので割愛

# LangChainを使用してすべてをまとめる時間です

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">必ずお読みください！非推奨警告を無視してください</h2>
            <span style="color:#900;">次のセルを実行すると、LangChainメモリのシンプルな使用方法に関するLangChainDeprecationWarningが表示されます。LangChainは、メモリに関する新しいアプローチへの移行を求めています。私はこれにかなり抵抗を感じています。新しいアプローチでは、LangGraphに移行し、そのエコシステムを深く理解する必要があります。LangGraphにはかなりの学習とコーディングが必要ですが、正直なところ、私たちのケースではあまりメリットがありません。<br/><br/>
                コースに組み込むかどうか、またどのように組み込むかについては検討中ですが、今のところは非推奨警告を無視してコードをそのまま使用してください。LangChainがConversationBufferMemoryをすぐに削除する予定はありません。
            </span>
        </td>
    </tr>
</table>

In [8]:
# OpenAIとの新しいチャットを作成します
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# チャットの会話メモリを設定します
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# RAGに使用されるベクターストアの抽象化
retriever = vectorstore.as_retriever()

# それをまとめる：GPT 4o-Mini LLM、ベクトルストア、メモリで会話チェーンをセットアップ
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

/tmp/ipykernel_30499/831555549.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [9]:
# 簡単な質問をしましょう
query = "Can you describe Insurellm in a few sentences"
result = conversation_chain.invoke({"question":query})
print(result["answer"])

Insurellm is an insurance tech startup founded in 2015 by Avery Lancaster, aimed at disrupting the insurance industry with innovative products. The company has grown to 200 employees and operates 12 offices across the US, offering four main software products: Carllm for auto insurance, Homellm for home insurance, Rellm for the reinsurance sector, and Marketllm, a marketplace connecting consumers with insurance providers. With over 300 clients worldwide, Insurellm is focused on transforming the insurance landscape through technology and innovation.


## GradioのChatインターフェイスを使用して、これを紹介します。
- LLMとのRAGチャットをプロトタイプする迅速かつ簡単な方法
- `Hi there.`, `What is Insurellm?`, `What did avery do before?`, `Does insurellm offer any products in the auto insurance space?` とでも聞いてみる。
- ポイントは、前回のソリューションはディクショナリのキー名に一致する文字列が質問文に含まれる必要があったが、ベクトル化とその格納によって、その制限が無くなった点。

In [10]:
# 関数のラッピング - 履歴の記憶はconversation_chainにあるため、Gradioのhistoryは使用されていない。

def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

In [11]:
# そしてGradioで：

view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
